In [1]:
from keras.models import load_model
import pandas as pd
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pickle
from keras.preprocessing.sequence import pad_sequences

maxlen = 130

Using TensorFlow backend.


In [2]:
loaded_model = load_model('sentimentClassifier.h5')

/home/suraj/Work/Hackathons-And-Challenges/Unisys/social-media-analytics-app/twitterEnv/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [3]:
stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

In [4]:
def load_tokenizer(filename):
    with open(filename, 'rb') as handle:
        tokenizer = pickle.load(handle)
    
    return tokenizer

loaded_tokenizer = load_tokenizer('tokenizer.pickle')

In [5]:
def getSentimentByAI(text, tokenizer, model):
    surajPred = pd.DataFrame({"id" : [1234], "review" : [text]})
    surajPred["review"]=surajPred.review.apply(lambda x: clean_text(x))
    list_sentences_test = surajPred["review"]
    list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
    X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
    
    prediction = loaded_model.predict(X_te)
    y_pred = (prediction > 0.5)
    return y_pred[0][0]

# Testing

In [6]:
getSentimentByAI("food stinked", loaded_tokenizer, loaded_model)

False

# Loading from the database

In [8]:
!pip3 install sqlalchemy

     |████████████████████████████████| 6.0 MB 1.5 MB/s eta 0:00:01
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.13-cp36-cp36m-linux_x86_64.whl size=1217180 sha256=0180846e4277f3bcb5c4fa4c3f2a6bded1eff825a51d65b5a86b70a1819f8a78
  Stored in directory: /home/suraj/.cache/pip/wheels/28/3e/f9/8eca04781258bb6956ffba37e4e6e6951e5b3a16d4494b91cb
Successfully built sqlalchemy


In [10]:
from sqlalchemy import create_engine

dbUrl = 'YOUR DATABASE URL'
engine = create_engine(dbUrl)

In [17]:
df = pd.read_sql_query('select * from twitterops_tweet',con=engine)

In [18]:
df.head()

,id,tweet_id,datetime,text,sentiment,customer_id,html
0,1,1221841391736606720,2020-01-27 17:04:23+00:00,@BLRAirport Two terrorists going to serve,neutral,1,"<blockquote class=""twitter-tweet""><p lang=""en""..."
1,2,1221840246045839361,2020-01-27 16:59:50+00:00,Bengaluru airport welcomed 33 million pax in 2...,neutral,2,"<blockquote class=""twitter-tweet""><p lang=""en""..."
2,3,1221838735014187008,2020-01-27 16:53:49+00:00,"To know more such interesting facts, follow th...",neutral,3,"<blockquote class=""twitter-tweet""><p lang=""en""..."
3,4,1221838463495917568,2020-01-27 16:52:45+00:00,@BLRAirport Excellent Staff Service by Krishna...,neutral,4,"<blockquote class=""twitter-tweet""><p lang=""en""..."
4,5,1221822560121184256,2020-01-27 15:49:33+00:00,Follow the BLR airport for such tasty facts.,neutral,5,"<blockquote class=""twitter-tweet""><p lang=""en""..."


In [22]:
tif = df.loc[0]['text']

In [27]:
len(df[df['sentiment'] == 'positive'])

6

In [26]:
len(df[df['sentiment'] == 'negative'])

1

In [23]:
getSentimentByAI(tif, loaded_tokenizer, loaded_model)

False

In [39]:
for i in range(len(df)):
    tif = df.loc[i]['text']
    sentiment = getSentimentByAI(tif, loaded_tokenizer, loaded_model)
    
    if(sentiment == True):
#         print('This is positive')
        (df['sentiment']).loc[i] = 'positive'
    elif(sentiment == False):
#         print('This is negative')
        (df['sentiment']).loc[i] = 'negative'

/home/suraj/Work/Hackathons-And-Challenges/Unisys/social-media-analytics-app/twitterEnv/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [45]:
df.head()

,id,tweet_id,datetime,text,sentiment,customer_id,html
0,1,1221841391736606720,2020-01-27 17:04:23+00:00,@BLRAirport Two terrorists going to serve,negative,1,"<blockquote class=""twitter-tweet""><p lang=""en""..."
1,2,1221840246045839361,2020-01-27 16:59:50+00:00,Bengaluru airport welcomed 33 million pax in 2...,positive,2,"<blockquote class=""twitter-tweet""><p lang=""en""..."
2,3,1221838735014187008,2020-01-27 16:53:49+00:00,"To know more such interesting facts, follow th...",negative,3,"<blockquote class=""twitter-tweet""><p lang=""en""..."
3,4,1221838463495917568,2020-01-27 16:52:45+00:00,@BLRAirport Excellent Staff Service by Krishna...,positive,4,"<blockquote class=""twitter-tweet""><p lang=""en""..."
4,5,1221822560121184256,2020-01-27 15:49:33+00:00,Follow the BLR airport for such tasty facts.,negative,5,"<blockquote class=""twitter-tweet""><p lang=""en""..."


In [41]:
len(df[df['sentiment'] == 'positive'])

40

In [42]:
len(df[df['sentiment'] == 'negative'])

37

In [43]:
len(df)

77

In [44]:
df.to_sql('temp_table', engine, if_exists='replace')

In [50]:
queryStr = "UPDATE temp_table set sentiment = \""+"positive"+"\" where tweet_id = "+str(1234)+";"
queryStr

'UPDATE temp_table set sentiment = "positive" where tweet_id = 1234;'

In [66]:
import psycopg2

try:
    connection = psycopg2.connect(user="USERNMAE HERE",
                                  password="PASSWORD HERE",
                                  host="HOST HERE",
                                  port="5432",
                                  database="ddutv3dsi8c1se USE YOUR OWN DATABASE")
    cursor = connection.cursor()
    record = cursor.fetchone()
    print(record)
    
    for i in range(len(df)):
        sentiment = (df.iloc[i])["sentiment"]
        cust = (df.loc[i])["customer_id"]
        sql_update_query = """Update twitterops_tweet set sentiment = %s where customer_id = %s"""
        cursor.execute(sql_update_query, (sentiment, cust))
        connection.commit()
        count = cursor.rowcount
        print(count, "Record Updated successfully ")
        
except (Exception, psycopg2.Error) as error :
    print ("Error while fetching data from PostgreSQL", error)

finally:
    #closing database connection.
    if(connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

Error while fetching data from PostgreSQL no results to fetch
PostgreSQL connection is closed


In [63]:
(df.iloc[0])["sentiment"]

'negative'

In [69]:
import psycopg2

def updateTable(customer_id, sentiment):
    try:
        connection = psycopg2.connect(user="USER",
                                  password="PASS",
                                  host="HOST",
                                  port="5432",
                                  database="DB")
        cursor = connection.cursor()

        print("Table Before updating record ")
        sql_select_query = """select * from twitterops_tweet where customer_id = %s"""
        cursor.execute(sql_select_query, (customer_id, ))
        record = cursor.fetchone()
        print(record)

        # Update single record now
        sql_update_query = """Update twitterops_tweet set sentiment = %s where customer_id = %s"""
        cursor.execute(sql_update_query, (sentiment, customer_id))
        connection.commit()
        count = cursor.rowcount
        print(count, "Record Updated successfully ")

        print("Table After updating record ")
        sql_select_query = """select * from twitterops_tweet where customer_id = %s"""
        cursor.execute(sql_select_query, (customer_id,))
        record = cursor.fetchone()
        print(record)

    except (Exception, psycopg2.Error) as error:
        print("Error in update operation", error)

    finally:
        # closing database connection.
        if (connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")


# updateTable(3, "positive")

Table Before updating record 
(3, '1221838735014187008', datetime.datetime(2020, 1, 27, 16, 53, 49, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 'To know more such interesting facts, follow the B L Rairport', 'negative', 3, '<blockquote class="twitter-tweet"><p lang="en" dir="ltr">To know more  such interesting facts, follow the <a href="https://twitter.com/hashtag/BLRairport?src=hash&amp;ref_src=twsrc%5Etfw">#BLRairport</a> <a href="https://t.co/vMqx700lqi">pic.twitter.com/vMqx700lqi</a></p>&mdash; BLR Airport (@BLRAirport) <a href="https://twitter.com/BLRAirport/status/1211649540576595969?ref_src=twsrc%5Etfw">December 30, 2019</a></blockquote>\n<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>\n')
1 Record Updated successfully 
Table After updating record 
(3, '1221838735014187008', datetime.datetime(2020, 1, 27, 16, 53, 49, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 'To know more such interesting facts, follow t

In [71]:
type((df.iloc[0])["customer_id"])

numpy.int64

In [72]:
(df.iloc[i])["customer_id"]

12

In [74]:
type(int((df.iloc[i])["customer_id"]))

int

In [75]:
for i in range(len(df)):
    updateTable(int((df.iloc[i])["customer_id"]), str((df.iloc[i])["sentiment"]))

Table Before updating record 
(1, '1221841391736606720', datetime.datetime(2020, 1, 27, 17, 4, 23, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), '@BLRAirport Two terrorists going to serve', 'negative', 1, '<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Two terrorists going to serve terrorist</p>&mdash; Virendra Rawat (@Virendraahd1) <a href="https://twitter.com/Virendraahd1/status/1221841391736606720?ref_src=twsrc%5Etfw">January 27, 2020</a></blockquote>\n<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>\n')
1 Record Updated successfully 
Table After updating record 
(1, '1221841391736606720', datetime.datetime(2020, 1, 27, 17, 4, 23, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), '@BLRAirport Two terrorists going to serve', 'negative', 1, '<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Two terrorists going to serve terrorist</p>&mdash; Virendra Rawat (@Virendraahd1) <a href="https://twitter.com/Vire

Table Before updating record 
(5, '1221822560121184256', datetime.datetime(2020, 1, 27, 15, 49, 33, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 'Follow the BLR airport for such tasty facts.', 'positive', 5, '<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Follow the BLR airport for such tasty facts. <a href="https://t.co/WNaYFvs8yR">pic.twitter.com/WNaYFvs8yR</a></p>&mdash; BLR Airport (@BLRAirport) <a href="https://twitter.com/BLRAirport/status/1212601517343031296?ref_src=twsrc%5Etfw">January 2, 2020</a></blockquote>\n<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>\n')
1 Record Updated successfully 
Table After updating record 
(5, '1221822560121184256', datetime.datetime(2020, 1, 27, 15, 49, 33, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 'Follow the BLR airport for such tasty facts.', 'negative', 5, '<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Follow the BLR airport for such tasty facts. 

2 Record Updated successfully 
Table After updating record 
(10, '1221806405646077952', datetime.datetime(2020, 1, 27, 14, 45, 21, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), "@BLRAviation @khabri_lal @BLRAirport I think it's a matter of time, also feel that they are pre emptying 6e launch potentially as", 'positive', 9, '<blockquote class="twitter-tweet"><p lang="en" dir="ltr">I think it&#39;s a matter of time, also feel that they are pre emptying 6e launch potentially as well....</p>&mdash; Vikram (@vikramv1) <a href="https://twitter.com/vikramv1/status/1221806405646077952?ref_src=twsrc%5Etfw">January 27, 2020</a></blockquote>\n<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>\n')
PostgreSQL connection is closed
Table Before updating record 
(56, '1221843217118826504', datetime.datetime(2020, 1, 27, 17, 11, 38, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 'A Mysterious Discovery on Western Global Airlines via', 'n

2 Record Updated successfully 
Table After updating record 
(14, '1221678985408417792', datetime.datetime(2020, 1, 27, 6, 19, 2, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), "iTWEET:On a busy day, Bengaluru Airport is all about Stress Relief! As you stand in the long queues to enter &amp; as you stand in longer &amp; slower queues to pass through security, it's S T R E S S! And once through: K I A L @BLRAirport", 'positive', 12, '<blockquote class="twitter-tweet"><p lang="en" dir="ltr">iTWEET:On a busy day, <a href="https://twitter.com/hashtag/BengaluruAirport?src=hash&amp;ref_src=twsrc%5Etfw">#BengaluruAirport</a> is all about <a href="https://twitter.com/hashtag/StressRelief?src=hash&amp;ref_src=twsrc%5Etfw">#StressRelief</a>! As you stand in the long queues to enter &amp; as you stand in longer &amp; slower queues to pass through security, it&#39;s <a href="https://twitter.com/hashtag/STRESS?src=hash&amp;ref_src=twsrc%5Etfw">#STRESS</a>! And once through: <a href="h

Table Before updating record 
(23, '1221843000109731841', datetime.datetime(2020, 1, 27, 17, 10, 46, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), '@reedahkh Ai until it kills like 100 before they will start running upandan. Even at that some selfish men will be finding contract of supply at the ministry. This is a very serious issue that needs urgent attention at our airports to be', 'negative', 20, '<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Ai until it kills like 100 before they will start running upandan. Even at that some selfish men will be finding contract of supply at the ministry. This is a very serious issue that needs urgent attention at our airports to be precise</p>&mdash; Muhammed Tijjani (@MuhammedTijjan7) <a href="https://twitter.com/MuhammedTijjan7/status/1221843000109731841?ref_src=twsrc%5Etfw">January 27, 2020</a></blockquote>\n<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>\n')
1 Record Updated succ

...

Table Before updating record 
(60, '1221840796082753537', datetime.datetime(2020, 1, 27, 17, 2, 1, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 'Throwback to that scheme teamcanon @BoeingDefense @AvNewsMag', 'negative', 56, '<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Throwback to that scheme 😍 <a href="https://twitter.com/hashtag/avgeek?src=hash&amp;ref_src=twsrc%5Etfw">#avgeek</a> <a href="https://twitter.com/hashtag/avporn?src=hash&amp;ref_src=twsrc%5Etfw">#avporn</a> <a href="https://twitter.com/hashtag/aviation?src=hash&amp;ref_src=twsrc%5Etfw">#aviation</a> <a href="https://twitter.com/hashtag/aviationphotography?src=hash&amp;ref_src=twsrc%5Etfw">#aviationphotography</a> <a href="https://twitter.com/hashtag/aircraft?src=hash&amp;ref_src=twsrc%5Etfw">#aircraft</a> <a href="https://twitter.com/hashtag/airshow?src=hash&amp;ref_src=twsrc%5Etfw">#airshow</a> <a href="https://twitter.com/hashtag/airtattoo?src=hash&amp;ref_src=twsrc%5Etfw">#airtattoo</a> <a

Table Before updating record 
(62, '1221840743070928898', datetime.datetime(2020, 1, 27, 17, 1, 48, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 'Throwback to that scheme teamcanon @BoeingDefense @AvNewsMag', 'negative', 58, '<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Throwback to that scheme 😍 <a href="https://twitter.com/hashtag/avgeek?src=hash&amp;ref_src=twsrc%5Etfw">#avgeek</a> <a href="https://twitter.com/hashtag/avporn?src=hash&amp;ref_src=twsrc%5Etfw">#avporn</a> <a href="https://twitter.com/hashtag/aviation?src=hash&amp;ref_src=twsrc%5Etfw">#aviation</a> <a href="https://twitter.com/hashtag/aviationphotography?src=hash&amp;ref_src=twsrc%5Etfw">#aviationphotography</a> <a href="https://twitter.com/hashtag/aircraft?src=hash&amp;ref_src=twsrc%5Etfw">#aircraft</a> <a href="https://twitter.com/hashtag/airshow?src=hash&amp;ref_src=twsrc%5Etfw">#airshow</a> <a href="https://twitter.com/hashtag/airtattoo?src=hash&amp;ref_src=twsrc%5Etfw">#airtattoo</a> <

1 Record Updated successfully 
Table After updating record 
(63, '1221840670190530566', datetime.datetime(2020, 1, 27, 17, 1, 31, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 'Throwback to that scheme teamcanon @BoeingDefense @AvNewsMag', 'positive', 59, '<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Throwback to that scheme 😍 <a href="https://twitter.com/hashtag/avgeek?src=hash&amp;ref_src=twsrc%5Etfw">#avgeek</a> <a href="https://twitter.com/hashtag/avporn?src=hash&amp;ref_src=twsrc%5Etfw">#avporn</a> <a href="https://twitter.com/hashtag/aviation?src=hash&amp;ref_src=twsrc%5Etfw">#aviation</a> <a href="https://twitter.com/hashtag/aviationphotography?src=hash&amp;ref_src=twsrc%5Etfw">#aviationphotography</a> <a href="https://twitter.com/hashtag/aircraft?src=hash&amp;ref_src=twsrc%5Etfw">#aircraft</a> <a href="https://twitter.com/hashtag/airshow?src=hash&amp;ref_src=twsrc%5Etfw">#airshow</a> <a href="https://twitter.com/hashtag/airtattoo?src=hash&amp;ref_src

Table Before updating record 
(69, '1221842995776897024', datetime.datetime(2020, 1, 27, 17, 10, 45, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), '@pizzahut I challenge u to stop selling', 'negative', 65, '<blockquote class="twitter-tweet"><p lang="en" dir="ltr">I challenge u to stop selling pizzas</p>&mdash; ごきぶり (@GogetaKurasaki) <a href="https://twitter.com/GogetaKurasaki/status/1221842995776897024?ref_src=twsrc%5Etfw">January 27, 2020</a></blockquote>\n<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>\n')
1 Record Updated successfully 
Table After updating record 
(69, '1221842995776897024', datetime.datetime(2020, 1, 27, 17, 10, 45, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), '@pizzahut I challenge u to stop selling', 'positive', 65, '<blockquote class="twitter-tweet"><p lang="en" dir="ltr">I challenge u to stop selling pizzas</p>&mdash; ごきぶり (@GogetaKurasaki) <a href="https://twitter.com/GogetaKurasaki/status/

Table Before updating record 
(76, '1221842200868290564', datetime.datetime(2020, 1, 27, 17, 7, 36, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), '@pizzahut Havent had any Pizza Hut for almost a year! The people here are rude and restaurant is', 'negative', 72, '<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Haven’t had any Pizza Hut for almost a year! The people here are rude and restaurant is disgusting.</p>&mdash; Aileen Orduño (@AileenOrduno) <a href="https://twitter.com/AileenOrduno/status/1221842200868290564?ref_src=twsrc%5Etfw">January 27, 2020</a></blockquote>\n<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>\n')
1 Record Updated successfully 
Table After updating record 
(76, '1221842200868290564', datetime.datetime(2020, 1, 27, 17, 7, 36, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), '@pizzahut Havent had any Pizza Hut for almost a year! The people here are rude and restaurant is', 'positive', 72, 

In [76]:
df_check = pd.read_sql_query('select * from twitterops_tweet',con=engine)
df_check.head()

,id,tweet_id,datetime,text,sentiment,customer_id,html
0,3,1221838735014187008,2020-01-27 16:53:49+00:00,"To know more such interesting facts, follow th...",negative,3,"<blockquote class=""twitter-tweet""><p lang=""en""..."
1,4,1221838463495917568,2020-01-27 16:52:45+00:00,@BLRAirport Excellent Staff Service by Krishna...,positive,4,"<blockquote class=""twitter-tweet""><p lang=""en""..."
2,5,1221822560121184256,2020-01-27 15:49:33+00:00,Follow the BLR airport for such tasty facts.,negative,5,"<blockquote class=""twitter-tweet""><p lang=""en""..."
3,9,1221806466983530496,2020-01-27 14:45:36+00:00,@BLRAviation @khabri_lal @BLRAirport Pre empti...,positive,9,"<blockquote class=""twitter-tweet""><p lang=""en""..."
4,14,1221678985408417792,2020-01-27 06:19:02+00:00,"iTWEET:On a busy day, Bengaluru Airport is all...",negative,12,"<blockquote class=""twitter-tweet""><p lang=""en""..."


In [77]:
len(df_check)

77